## Summary: Band Gap Prediction (Materials Project) — ANN vs Tree Ensembles
> **Razieh Morad**

- **Problem:** Predict PBE band gap (eV) for stable binary semiconductors.
- **Data:** Materials Project (MP API), one representative per formula (N = …).
- **Approach:** 
>- Composition featurization: matminer 
>- Artificial Neural Network (ANN) vs. RF/XGB/GBR 
>- Split: IID (stratified by binned band gap)
- **Metrics:** MAE / RMSE / R² on **test**; parity plots; XGB feature importance.
- **Takeaway:** _Fill after results_: e.g., “XGB wins on IID (MAE A→B, R² C→D vs. Ridge/ANN). ANN competitive, but needs more data/regularization.”


In [ ]:
# ======================
# 0) Imports & Config
# ======================
import os, math, json, warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from mp_api.client import MPRester
from pymatgen.core import Composition

from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.composition import (
    ElementProperty, OxidationStates, AtomicOrbitals, BandCenter
)

from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

warnings.filterwarnings("ignore")
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Repro / paths
RNG_SEED = 42
np.random.seed(RNG_SEED)
ROOT = Path.cwd()
FIGS = ROOT.parent / "reports" / "figures" if (ROOT.name == "notebooks") else ROOT / "reports" / "figures"
FIGS.mkdir(parents=True, exist_ok=True)

print("Working dir:", ROOT)

## 1. Load datasets of binary semiconductors from Materials Project

In [ ]:
# ======================
# 1. SET API KEY
# ======================
api_key = "VzNVmBMga05iZbWxZz3XFrV74z9ih3jE"


In [ ]:
valid_fields = [
    "material_id","formula_pretty","elements","nsites","nelements","density","volume",
    "energy_per_atom","formation_energy_per_atom","energy_above_hull","band_gap","is_gap_direct",
    "total_magnetization","bulk_modulus","shear_modulus","homogeneous_poisson","efermi",
    "universal_anisotropy","n","e_total"
]

print("############  Fetching stable binary semiconductors… ")
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
        num_elements=[2, 2],
        energy_above_hull=(0, 0.10),
        is_metal=False,
        band_gap=(0.01, None),
        chunk_size=100,
        num_chunks=10,              # ↓ reduce to 3–5 if it's too slow
        fields=valid_fields,
    )

rows = []
for d in docs:
    rows.append({
        "material_id": d.material_id,
        "formula": d.formula_pretty,
        "elements": tuple(d.elements) if getattr(d, "elements", None) else None,
        "nsites": d.nsites,
        "nelements": d.nelements,
        "density": d.density,
        "volume": d.volume,
        "energy_per_atom": d.energy_per_atom,
        "formation_energy_per_atom": d.formation_energy_per_atom,
        "energy_above_hull": d.energy_above_hull,
        "band_gap": d.band_gap,
        "is_gap_direct": int(d.is_gap_direct) if d.is_gap_direct is not None else 0,
        "total_magnetization": d.total_magnetization,
        "bulk_modulus": d.bulk_modulus,
        "shear_modulus": d.shear_modulus,
        "poisson_ratio": d.homogeneous_poisson,
        "fermi_energy": d.efermi,
        "elastic_anisotropy": d.universal_anisotropy,
        "refractive_index": d.n,
        "dielectric_constant": d.e_total,
    })

df_raw = pd.DataFrame(rows)
print("Fetched:", df_raw.shape)
df_raw.head(3)

## 2. QA & deduplicate

- Keep one representative per formula (lowest `E_hull`, then smallest volume).  
- Drop broken rows / negative band gaps.


In [ ]:
df_raw = (
    df_raw.sort_values(["energy_above_hull", "volume"], ascending=[True, True])
          .drop_duplicates("formula", keep="first")
          .reset_index(drop=True)
)

df_raw = df_raw.dropna(subset=["band_gap"]).query("band_gap >= 0")
print("After dedup & QA:", df_raw.shape)
df_raw.describe(include="all").T.head(10)


## 3. Featurize (composition-only; fast & solid)

Using matminer to compute composition descriptors: **Magpie**, oxidation states, atomic orbitals, and band center.  
Keeping numeric columns for ML and carry a **meta** table (formula, elements) for slicing/plots later.


In [ ]:
# Composition objects
df_feat = df_raw.copy()
df_feat["composition"] = df_feat["formula"].apply(Composition)

featurizer = MultipleFeaturizer([
    ElementProperty.from_preset("magpie"),
    OxidationStates(),
    AtomicOrbitals(),
    BandCenter(),
])

# Compute features in-place (ignore per-row errors)
df_feat = featurizer.featurize_dataframe(df_feat, col_id="composition", ignore_errors=True)

# Keep meta for later slicing
meta = df_feat[["formula","elements"]].copy()

# Keep numeric features only (drop target later)
# meta holds non-numeric context (formula, elements) for plots/slicing/OOD masks.
df_num = df_feat.select_dtypes(include=[np.number]).copy()
df_num = df_num.dropna(subset=["band_gap"]).reset_index(drop=True)
meta = meta.loc[df_num.index].reset_index(drop=True)

print("Numeric feature table:", df_num.shape)
df_num.head(3)


## 4. Split (IID) with stratification, and preprocessing per model family

- **Trees**: impute missing values (no scaling needed).  
- **ANN**: impute + scale X, and scale **y** for stable training.  
- We stratify the split by **binned** band gap to match target distribution in train/test.


In [ ]:
# Separate X and y
X = df_num.drop(columns=["band_gap"])
y = df_num["band_gap"].values.astype(np.float32)

# Use indices to keep alignment with meta
idx = np.arange(len(X))
bins = pd.qcut(y, q=10, duplicates="drop")

idx_tr, idx_te = train_test_split(
    idx, test_size=0.2, random_state=RNG_SEED, stratify=bins
)

X_train = X.iloc[idx_tr]; X_test  = X.iloc[idx_te]
y_train = y[idx_tr];      y_test  = y[idx_te]
meta_te = meta.iloc[idx_te].reset_index(drop=True)

# Preprocessing
imp = SimpleImputer(strategy="median")
X_train_tree = imp.fit_transform(X_train)
X_test_tree  = imp.transform(X_test)

scaler_X = StandardScaler()
X_train_ann = scaler_X.fit_transform(X_train_tree)   # use imputed numeric arrays
X_test_ann  = scaler_X.transform(X_test_tree)

print("Train shapes — tree:", X_train_tree.shape, " | ann:", X_train_ann.shape)
print(f"y range (eV): {y.min():.2f}–{y.max():.2f}")



In [ ]:
import numpy as np

def chk(name, arr):
    arr = np.asarray(arr)
    print(f"{name:12s} shape={arr.shape}  finite={np.isfinite(arr).all()}  "
          f"n_nan={np.isnan(arr).sum()}  n_inf={np.isinf(arr).sum()}")

chk("X_train_tree", X_train_tree)
chk("X_test_tree",  X_test_tree)
chk("X_train_ann",  X_train_ann)
chk("X_test_ann",   X_test_ann)

print("y_train stats (eV): mean=%.3f std=%.3f" % (y_train.mean(), y_train.std()))
print("y_test  stats (eV): mean=%.3f std=%.3f" % (y_test.mean(),  y_test.std()))


In [ ]:
import numpy as np

def chk(name, arr):
    print(f"{name}: shape={arr.shape}  finite={np.isfinite(arr).all()}  "
          f"n_nan={np.isnan(arr).sum()}  n_inf={np.isinf(arr).sum()}")

chk("X_train_ann", X_train_ann)
chk("X_test_ann",  X_test_ann)

# If using StandardScaler, inspect its scales (std dev per feature)
try:
    print("min/std/median scaler_X.scale_:", np.min(scaler_X.scale_), np.max(scaler_X.scale_), np.median(scaler_X.scale_))
except Exception as e:
    print("No scaler_X.scale_ yet:", e)


In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Impute numeric features
imp = SimpleImputer(strategy="median")
X_train_imp = imp.fit_transform(X_train)
X_test_imp  = imp.transform(X_test)

# Drop zero-variance columns (prevents 1/0 in StandardScaler)
vt = VarianceThreshold(threshold=0.0)
X_train_vt = vt.fit_transform(X_train_imp)
X_test_vt  = vt.transform(X_test_imp)

# Scale for ANN
scaler_X = StandardScaler()
X_train_ann = scaler_X.fit_transform(X_train_vt)
X_test_ann  = scaler_X.transform(X_test_vt)

# Quick guards
assert np.isfinite(X_train_ann).all() and np.isfinite(X_test_ann).all(), "Non-finite values after scaling!"
print("✅ preprocessing OK —",
      "X_train_ann", X_train_ann.shape, "| X_test_ann", X_test_ann.shape,
      "| min scale", scaler_X.scale_.min())

## 5. ANN model (PyTorch) + K-fold CV (with **per-fold** y-scaler)

We scale **y** inside each fold to avoid leakage. Final test predictions are inverse-transformed with the best fold’s scaler.


In [ ]:
# ======================
# 5) ANN (PyTorch) + K-fold CV with PER-FOLD y-scaler
# ======================
# This cell trains an MLP (ANN) with:
# - input scaling already applied to X (X_train_ann / X_test_ann)
# - target scaling applied PER FOLD (no leakage)
# - early stopping via patience on validation loss
# - ReduceLROnPlateau scheduler for smoother convergence

import math
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

RNG_SEED = 42
torch.manual_seed(RNG_SEED)
np.random.seed(RNG_SEED)

# ----------------------
# ANN architecture  nn.Dropout(0.15)
# ----------------------
class BandGapANN(nn.Module):
    def __init__(self, d_in: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_in, 256), nn.ReLU(), nn.Dropout(0.05),
            nn.Linear(256, 128),  nn.ReLU(), nn.Dropout(0.05),
            nn.Linear(128,  64),  nn.ReLU(), nn.Dropout(0.05),
            nn.Linear(64, 1)
        )
    def forward(self, x): 
        return self.net(x)

# ----------------------
# One training run (given a train/val split, with y already scaled)
# ----------------------
def train_ann_epoched(Xtr, ytr_scaled, Xva, yva_scaled, n_epochs=100, batch_size=32, lr=5e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # numpy -> torch
    Xtr_t = torch.tensor(Xtr, dtype=torch.float32).to(device)
    ytr_t = torch.tensor(ytr_scaled, dtype=torch.float32).view(-1, 1).to(device)
    Xva_t = torch.tensor(Xva, dtype=torch.float32).to(device)
    yva_t = torch.tensor(yva_scaled, dtype=torch.float32).view(-1, 1).to(device)

    loader = DataLoader(TensorDataset(Xtr_t, ytr_t), batch_size=batch_size, shuffle=True)

    model = BandGapANN(Xtr.shape[1]).to(device)
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    crit = nn.MSELoss()
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=5)

    best_state, best_val, patience, max_patience = model.state_dict(), float("inf"), 0, 10
    train_losses, val_losses = [], []

    for epoch in range(n_epochs):
        # --- train ---
        model.train()
        running = 0.0
        for xb, yb in loader:
            opt.zero_grad()
            loss = crit(model(xb), yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            opt.step()
            running += loss.item() * xb.size(0)
        train_loss = running / len(loader.dataset)

        # --- validate ---
        model.eval()
        with torch.no_grad():
            val_loss = crit(model(Xva_t), yva_t).item()

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        sched.step(val_loss)

        # --- early stopping bookkeeping ---
        if val_loss < best_val:
            best_val = val_loss
            best_state = model.state_dict()
            patience = 0
        else:
            patience += 1
            if patience >= max_patience:
                break

    # restore best weights
    model.load_state_dict(best_state)
    model.eval()
    return model, train_losses, val_losses

# ----------------------
# K-fold wrapper with PER-FOLD y-scaler (no leakage)
# ----------------------
def kfold_ann_with_perfold_scaler(X_ann, y_ev, n_splits=5, n_epochs=100, batch_size=32):
    """
    X_ann : numpy array (imputed + scaled features for ANN)
    y_ev  : numpy array of band gaps in eV (UNSCALED)
    Returns: list of (model, y_scaler) for each fold, and a DataFrame with fold metrics.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RNG_SEED)
    fold_rows, fold_models = [], []

    for i, (tr, va) in enumerate(kf.split(X_ann), start=1):
        # ---- PER-FOLD y-scaler: fit on y_train_fold ONLY (prevents leakage)
        y_scaler = StandardScaler().fit(y_ev[tr].reshape(-1, 1))
        y_tr_scaled = y_scaler.transform(y_ev[tr].reshape(-1, 1)).ravel()
        y_va_scaled = y_scaler.transform(y_ev[va].reshape(-1, 1)).ravel()

        # ---- Train one ANN on this fold
        model, tr_loss, va_loss = train_ann_epoched(
            X_ann[tr], y_tr_scaled,
            X_ann[va], y_va_scaled,
            n_epochs=n_epochs, batch_size=batch_size, lr=1e-3
        )

        fold_models.append((model, y_scaler))

        # ---- Predict on validation fold, inverse-transform back to eV for honest metrics
        with torch.no_grad():
            preds_scaled = model(torch.tensor(X_ann[va], dtype=torch.float32)).cpu().numpy().ravel()
        preds_ev = y_scaler.inverse_transform(preds_scaled.reshape(-1, 1)).ravel()
        truth_ev = y_ev[va]

        mae  = mean_absolute_error(truth_ev, preds_ev)
        rmse = math.sqrt(mean_squared_error(truth_ev, preds_ev))
        r2   = r2_score(truth_ev, preds_ev)
        fold_rows.append(dict(fold=i, MAE=mae, RMSE=rmse, R2=r2))

        # (optional) quick curve
        plt.figure(figsize=(6,3))
        plt.plot(tr_loss, label="train"); plt.plot(va_loss, label="val")
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.legend()
        plt.title(f"ANN training curves (fold {i})"); plt.legend(); plt.tight_layout(); plt.show()

    cvdf = pd.DataFrame(fold_rows)
    print(cvdf)
    print(f"ANN CV — mean R²: {cvdf.R2.mean():.3f} ± {cvdf.R2.std():.3f}")
    return fold_models, cvdf

# ----------------------
# Run K-fold on your training split, then evaluate once on TEST
# ----------------------
print(" Training ANN with K-fold CV on training split…")
ann_models, ann_cv = kfold_ann_with_perfold_scaler(
    X_train_ann,  # features for ANN (already imputed+scaled)
    y_train,      # target in eV (UNSCALED here)
    n_splits=5, n_epochs=100, batch_size=32
)

# Pick best fold by R², use ITS y-scaler to map test preds back to eV
best_idx = ann_cv["R2"].idxmax()
best_ann, best_y_scaler = ann_models[best_idx]

with torch.no_grad():
    y_pred_test_scaled = best_ann(torch.tensor(X_test_ann, dtype=torch.float32)).cpu().numpy().ravel()
y_pred_test_ev = best_y_scaler.inverse_transform(y_pred_test_scaled.reshape(-1,1)).ravel()

mae  = mean_absolute_error(y_test, y_pred_test_ev)
rmse = math.sqrt(mean_squared_error(y_test, y_pred_test_ev))
r2   = r2_score(y_test, y_pred_test_ev)
print(f"ANN (TEST) — MAE {mae:.3f} | RMSE {rmse:.3f} | R² {r2:.3f}")


In [ ]:
def parity(y_true, y_pred, title):
    import numpy as np, matplotlib.pyplot as plt, os
    os.makedirs("reports/figures", exist_ok=True)
    lim = (0, max(np.max(y_true), np.max(y_pred)))
    plt.figure(figsize=(5,5))
    plt.scatter(y_true, y_pred, s=6, alpha=0.5)
    plt.plot(lim, lim, "--"); plt.xlim(lim); plt.ylim(lim)
    plt.xlabel("True band gap (eV)"); plt.ylabel("Predicted (eV)"); plt.title(title)
    plt.tight_layout(); plt.savefig("reports/figures/parity_ann.png", dpi=160); plt.show()

parity(y_test, y_pred_test_ev, "ANN")


In [ ]:
from pymatgen.core import Composition
HOLDOUT = {"Bi","Te"}

def has_holdout(formula): 
    try: return any(el.symbol in HOLDOUT for el in Composition(formula).elements)
    except: return False

mask_ood = meta["formula"].apply(has_holdout).values
X_ood = X.loc[mask_ood]

# transform with the SAME train preprocessors used for ANN
X_ood_imp = imp.transform(X_ood)
X_ood_vt  = vt.transform(X_ood_imp)          # if you used VarianceThreshold; else skip this line
X_ood_ann = scaler_X.transform(X_ood_vt if 'vt' in globals() else X_ood_imp)

with torch.no_grad():
    pred_ood_s = best_ann(torch.tensor(X_ood_ann, dtype=torch.float32)).cpu().numpy().ravel()
y_ood_pred = best_y_scaler.inverse_transform(pred_ood_s.reshape(-1,1)).ravel()
y_ood_true = y[mask_ood]

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
print(f"ANN OOD {sorted(HOLDOUT)} — MAE {mean_absolute_error(y_ood_true, y_ood_pred):.3f} | "
      f"RMSE {math.sqrt(mean_squared_error(y_ood_true, y_ood_pred)):.3f} | "
      f"R² {r2_score(y_ood_true, y_ood_pred):.3f}")


## 6. Classical models (trees) on imputed features

- Trees don’t need scaling; we use the imputed numeric arrays.  
- We compute comparable test metrics.


In [ ]:

# --- Trees preprocessing ---
imp = SimpleImputer(strategy="median")
X_train_tree = imp.fit_transform(X_train)
X_test_tree  = imp.transform(X_test)

# Optional but recommended (prevents 1/0 std issues later)
vt = VarianceThreshold(threshold=0.0)
X_train_tree = vt.fit_transform(X_train_tree)
X_test_tree  = vt.transform(X_test_tree)

models = {
    "Random Forest": RandomForestRegressor(n_estimators=300, max_depth=12, random_state=0, n_jobs=-1),
    "XGBoost": XGBRegressor(
        n_estimators=400, learning_rate=0.06, max_depth=8,
        subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
        tree_method="hist", random_state=0, n_jobs=-1
    ),
    "Gradient Boosting": GradientBoostingRegressor(
        n_estimators=400, learning_rate=0.06, max_depth=6, random_state=0
    ),
}

results, preds = [], {}
for name, model in models.items():
    model.fit(X_train_tree, y_train)          # <-- eV target
    yp = model.predict(X_test_tree)           # <-- on matching tree features
    preds[name] = yp
    mae  = mean_absolute_error(y_test, yp)
    rmse = math.sqrt(mean_squared_error(y_test, yp))
    r2   = r2_score(y_test, yp)
    results.append(dict(Model=name, MAE=mae, RMSE=rmse, R2=r2))

# Add ANN (your existing y_pred_ann in eV)
results.append(dict(Model="ANN",
                    MAE=mean_absolute_error(y_test, y_pred_ann),
                    RMSE=math.sqrt(mean_squared_error(y_test, y_pred_ann)),
                    R2=r2_score(y_test, y_pred_ann)))
results_df = pd.DataFrame(results).sort_values("R2", ascending=False).reset_index(drop=True)
display(results_df)

## 7. Parity plots (saved) & XGB feature importance


In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# ensure output folder exists (use your existing FIGS if defined)
try:
    FIGS
except NameError:
    FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def parity_grid(y_true, pred_dict, layout="2x2", fname="parity_grid.png"):
    """
    layout: "2x2" for a 2-by-2 grid, or "row" for all side-by-side in one row.
    """
    names = list(pred_dict.keys())
    preds = [pred_dict[k] for k in names]

    # consistent axes across all panels
    max_val = max(float(np.max(y_true)), max(float(np.max(p)) for p in preds))
    lim = (0, max_val)

    if layout == "row":
        nrows, ncols = 1, len(names)
    else:  # "2x2" (default) or auto for up to 4 models
        ncols = 2
        nrows = math.ceil(len(names) / ncols)

    fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 5*nrows), squeeze=False)
    axes_flat = axes.ravel()

    for i, (name, yp) in enumerate(zip(names, preds)):
        ax = axes_flat[i]
        ax.scatter(y_true, yp, s=6, alpha=0.5)
        ax.plot(lim, lim, "--")
        ax.set_xlim(lim); ax.set_ylim(lim)
        ax.set_xlabel("True band gap (eV)")
        ax.set_ylabel("Predicted (eV)")
        ax.set_title(name)

    # hide any unused subplots
    for j in range(i+1, len(axes_flat)):
        axes_flat[j].axis("off")

    fig.tight_layout()
    out = FIGS / fname
    fig.savefig(out, dpi=160)
    plt.show()
    print("Saved:", out)

# Usage:
# 2x2 grid
#parity_grid(y_test, preds, layout="2x2", fname="parity_grid_2x2.png")

# side-by-side in one row
parity_grid(y_test, preds, layout="row", fname="parity_grid_row.png")


In [ ]:
# XGB feature importance
xgb_model = models["XGBoost"]   # already fitted
imps = xgb_model.feature_importances_
feat_names = X.columns.tolist()   # corresponds to columns used before impute

top = np.argsort(imps)[::-1][:15]
plt.figure(figsize=(12,5))
plt.bar(range(len(top)), imps[top])
plt.xticks(range(len(top)), [feat_names[i] for i in top], rotation=60, ha="right")
plt.title("Top 15 Feature Importances (XGBoost)")
plt.tight_layout()
out = FIGS / "xgb_feature_importance.png"
plt.savefig(out, dpi=160); plt.show()
print("Saved:", out)


## 8. (Optional) OOD hold-out by elements (chemistry shift)

Train on compounds **without** selected elements and test on those **with** them.  
This simulates a domain shift (e.g., cold-start chemistries).


In [ ]:
print("type(y):", type(y), "shape:", getattr(y, "shape", None))
X_full = df_num.drop(columns=["band_gap"]).copy()
y_full = df_num["band_gap"].values.astype(np.float32)
meta_full = meta.copy()  # must correspond 1:1 with df_num

assert len(X_full) == len(y_full) == len(meta_full), "lengths must match"

In [ ]:
from pymatgen.core import Composition

HOLDOUT = {"Bi","Te"}  # adjust if too small/large

def has_holdout(formula: str) -> bool:
    try:
        return any(el.symbol in HOLDOUT for el in Composition(formula).elements)
    except Exception:
        return False

mask_ood = meta_full["formula"].apply(has_holdout).values  # length == len(X_full)

# In-domain (train) vs OOD (test)
X_tr_ood, y_tr_ood = X_full.loc[~mask_ood], y_full[~mask_ood]
X_te_ood, y_te_ood = X_full.loc[ mask_ood], y_full[ mask_ood]

print("OOD sizes:", X_tr_ood.shape, X_te_ood.shape)

from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import math

# Impute & drop zero-variance cols using ONLY in-domain data
imp_ood = SimpleImputer(strategy="median").fit(X_tr_ood)
X_tr_imp = imp_ood.transform(X_tr_ood)
X_te_imp = imp_ood.transform(X_te_ood)

vt_ood = VarianceThreshold(0.0).fit(X_tr_imp)
X_tr_vt = vt_ood.transform(X_tr_imp)
X_te_vt = vt_ood.transform(X_te_imp)

# Train on in-domain, test on OOD
xgb_ood = XGBRegressor(
    n_estimators=400, learning_rate=0.06, max_depth=8,
    subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
    tree_method="hist", random_state=0, n_jobs=-1
).fit(X_tr_vt, y_tr_ood)

y_pred_ood = xgb_ood.predict(X_te_vt)
mae  = mean_absolute_error(y_te_ood, y_pred_ood)
rmse = math.sqrt(mean_squared_error(y_te_ood, y_pred_ood))
r2   = r2_score(y_te_ood, y_pred_ood)
print(f"XGB OOD {sorted(HOLDOUT)} — MAE {mae:.3f} | RMSE {rmse:.3f} | R² {r2:.3f}")


## 9. (Optional) Save artifacts

Only if you want a small inference demo later.


In [ ]:
import joblib, os
os.makedirs("models", exist_ok=True)
joblib.dump(models["XGBoost"], "models/xgb_bandgap.joblib")
np.savez("models/ann_scalers.npz",
         imp_statistics=imp.statistics_,   # imputer stats
         scaler_X_mean=scaler_X.mean_, scaler_X_scale=scaler_X.scale_)
print("Saved models/… (XGB + scalers)")
